In [1]:
import pandas as pd
import numpy as np
from pymongo.mongo_client import MongoClient
from datetime import datetime
import asyncio
import time
import math
import json
import requests
import ipaddress
import uuid
from kafka import KafkaConsumer, KafkaProducer
import  sys 
import six
if sys.version_info >= (3, 12, 0):
    sys.modules['kafka.vendor.six.moves'] = six.moves

def MatchTrades_sliding_window():

    obj_MT  = None

    response = requests.get("https://www.binance.com/api/v1/aggTrades?limit=1&symbol=BTCEUR").json()

    data_MT = {"timestamp":response[0]["T"],
                    "amount":response[0]["q"],
                    "price":response[0]["p"]}
    
    # post to kafka #
    try:
        producer = KafkaProducer(bootstrap_servers="localhost:9092")
        producer.send('MatchTrades-data', key=bytes(str(uuid.uuid4()), 'utf-8'), value= bytes( str( json.dumps(data_MT) ),'utf-8' ))
        producer.close()
    except:
        pass
 
    time.sleep(0.01)

In [2]:
#MatchTrades_binanceAPI()
%timeit -n 1 -r 100 MatchTrades_sliding_window()

458 ms ± 115 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [3]:
import pandas as pd
import numpy as np
from pymongo.mongo_client import MongoClient
from datetime import datetime
import asyncio
import time
import math
import json
import uuid
import asyncio
from tokenize import group
import requests
from kafka import KafkaConsumer, KafkaProducer
import  sys 
import six
if sys.version_info >= (3, 12, 0):
    sys.modules['kafka.vendor.six.moves'] = six.moves

# --------------------------------------------------------------- #
# calculate attributes from the lob data
# --------------------------------------------------------------- #

def lob_data_sliding_window():

    try: 
        response = requests.get("https://www.binance.com/api/v3/depth?symbol=BTCEUR&limit=200").json()
        data_LOB = {"timestamp":time_now,
                    "bids":response.get("bids"),
                    "asks":response.get("asks")}
    except:
        pass


    try:     
        # send data throught the kafka stream
        producer = KafkaProducer(bootstrap_servers="localhost:9092")
        producer.send('Lob-data', key=bytes(str(uuid.uuid4()), 'utf-8'), value= bytes( str( json.dumps(data_LOB) ),'utf-8' ))
        producer.close()

    except:
        pass

    time.sleep(0.01)
    # ------------- #


In [4]:
%timeit -n 1 -r 100  lob_data_sliding_window()

453 ms ± 127 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)
